In [44]:
import spacy
from spacy.tokens import DocBin
from spacy.training import Example
from spacy.util import Config
from spacy.training import Corpus
import random
from tqdm import tqdm
import pandas as pd 

In [46]:
nlp_fr = spacy.load("fr_core_news_md")
print("spacy model charge avec succes")

spacy model charge avec succes


In [48]:
texte_simple = '''Je voudrais aller de Ermont à Sannois. ...'''
doc = nlp_fr(texte_simple)
spacy.displacy.render(doc, style="ent", jupyter=True)

In [52]:
# Charger le fichier CSV
df = pd.read_csv('../../dataset/raw/initial_training_data.csv')
print("\nfichier csv charge avec succes\n")


fichier csv charge avec succes



In [54]:
# premieres lignes du dataset pour verifier le chargement
print(df.head())

                                            sentence  entity_start  \
0  Quels sites touristiques visiter en allant de ...            46   
1  Quels sites touristiques visiter en allant de ...            54   
2  Combien de temps faut-il pour aller de PARIS à...            39   
3  Combien de temps faut-il pour aller de PARIS à...            47   
4  Quels sont les moyens de transport les plus éc...            70   

   entity_end  entity_type  
0          51    DEPARTURE  
1          63  DESTINATION  
2          44    DEPARTURE  
3          51  DESTINATION  
4          75    DEPARTURE  


In [56]:
# on cree un conteneur pour stocker les documents au format spaCy
db = DocBin()
print("\nconteneur DocBin cree avec succes\n")


conteneur DocBin cree avec succes



In [60]:
# on melange les donnees pour éviter tout biais dans l'apprentissage
df = df.sample(frac=1).reset_index(drop=True)
print("\ndonnees melangees avec succes\n")


donnees melangees avec succes



In [62]:
# on convertit les donnees au format spaCy
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    text = row['sentence']
    doc = nlp_fr.make_doc(text)  # on cree un document spaCy a partir d'un texte
    ents = []
    
    # on recupere les infos des entites
    start = row['entity_start']
    end = row['entity_end']
    label = row['entity_type']
    
    # on cree d'une entite spaCy a partir des coordonnees de debut et de fin
    span = doc.char_span(start, end, label=label, alignment_mode="contract")
    if span is not None:
        ents.append(span)
    
    doc.ents = ents  # on assigne les entites au document
    db.add(doc)  # on ajoute le document au DobBin

100%|██████████| 499000/499000 [04:56<00:00, 1682.23it/s]


In [64]:
# enregistre les donnes converties au format spaCy
db.to_disk("../../dataset/processed/training_data.spacy")
print("\nDonnées converties et sauvegardées au format spaCy\n")


Données converties et sauvegardées au format spaCy



In [66]:
# ajout du composant NER au modele actuel
ner = nlp_fr.get_pipe("ner")

In [70]:
# ajoute de nouveaux labels d'entites a partir des donnees
for _, row in df.iterrows():
    ner.add_label(row['entity_type'])
print("\nles nouvelles labels NER ont ete ajoutees au modele\n")


les nouvelles labels NER ont ete ajoutees au modele



In [72]:
import os

# creation du fichier de configuration
os.system('python -m spacy init config ./training/config.cfg --lang fr --pipeline ner --optimize efficiency --force')
print("\nfichier configuration genere avec succes.\n")


fichier configuration genere avec succes.



In [ ]:
# entrainement du modele
os.system('python -m spacy train ./training/config.cfg --output ../../models/saved_models --paths.train ../../dataset/processed/training_data.spacy --paths.dev ../../dataset/processed/training_data.spacy')
print("\nEntraînement du modèle terminé.\n")